# Recurrent neural networks

Sa nakaraang module, tinalakay natin ang mga mayamang semantikong representasyon ng teksto. Ang arkitektura na ginagamit natin ay kumukuha ng pinagsama-samang kahulugan ng mga salita sa isang pangungusap, ngunit hindi nito isinasaalang-alang ang **pagkakasunod-sunod** ng mga salita, dahil ang operasyon ng pagsasama-sama na sumusunod sa mga embeddings ay nag-aalis ng impormasyong ito mula sa orihinal na teksto. Dahil hindi kayang kumatawan ng mga modelong ito sa pagkakasunod-sunod ng mga salita, hindi nila malulutas ang mas kumplikado o malalabong gawain tulad ng pagbuo ng teksto o pagsagot sa mga tanong.

Upang makuha ang kahulugan ng isang sunod-sunod na teksto, gagamit tayo ng arkitektura ng neural network na tinatawag na **recurrent neural network**, o RNN. Kapag gumagamit ng RNN, ipinapasa natin ang ating pangungusap sa network nang paisa-isang token, at ang network ay gumagawa ng isang **estado**, na pagkatapos ay ipinapasa muli sa network kasama ang susunod na token.

![Larawan na nagpapakita ng halimbawa ng pagbuo ng recurrent neural network.](../../../../../translated_images/tl/rnn.27f5c29c53d727b5.webp)

Sa ibinigay na input sequence ng mga token $X_0,\dots,X_n$, ang RNN ay lumilikha ng isang sunod-sunod na mga neural network block, at sinasanay ang sequence na ito mula simula hanggang dulo gamit ang backpropagation. Ang bawat network block ay tumatanggap ng pares $(X_i,S_i)$ bilang input, at gumagawa ng $S_{i+1}$ bilang resulta. Ang huling estado $S_n$ o output $Y_n$ ay ipinapasa sa isang linear classifier upang makabuo ng resulta. Ang lahat ng network block ay may parehong weights, at sinasanay mula simula hanggang dulo gamit ang isang backpropagation pass.

> Ang figure sa itaas ay nagpapakita ng recurrent neural network sa unrolled form (sa kaliwa), at sa mas compact na recurrent representation (sa kanan). Mahalagang maunawaan na ang lahat ng RNN Cells ay may parehong **shareable weights**.

Dahil ang mga state vector $S_0,\dots,S_n$ ay ipinapasa sa network, ang RNN ay may kakayahang matutunan ang mga sunod-sunod na dependencies sa pagitan ng mga salita. Halimbawa, kapag ang salitang *hindi* ay lumitaw sa isang bahagi ng sequence, maaari nitong matutunan na i-negate ang ilang elemento sa loob ng state vector.

Sa loob, ang bawat RNN cell ay naglalaman ng dalawang weight matrices: $W_H$ at $W_I$, at bias $b$. Sa bawat hakbang ng RNN, sa ibinigay na input $X_i$ at input state $S_i$, ang output state ay kinakalkula bilang $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, kung saan ang $f$ ay isang activation function (madalas $\tanh$).

> Para sa mga problema tulad ng pagbuo ng teksto (na tatalakayin natin sa susunod na unit) o machine translation, nais din nating makakuha ng ilang output value sa bawat hakbang ng RNN. Sa kasong ito, mayroon ding isa pang matrix $W_O$, at ang output ay kinakalkula bilang $Y_i=f(W_O\times S_i+b_O)$.

Tingnan natin kung paano makakatulong ang recurrent neural networks sa pag-classify ng ating news dataset.

> Para sa sandbox environment, kailangan nating patakbuhin ang sumusunod na cell upang matiyak na ang kinakailangang library ay naka-install, at ang data ay na-prefetch. Kung ikaw ay nagpa-patakbo nang lokal, maaari mong laktawan ang sumusunod na cell.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Kapag nagsasanay ng malalaking modelo, maaaring maging problema ang alokasyon ng GPU memory. Kailangan din nating mag-eksperimento sa iba't ibang laki ng minibatch upang magkasya ang data sa ating GPU memory, habang nananatiling mabilis ang pagsasanay. Kung pinapatakbo mo ang code na ito sa sarili mong GPU machine, maaari kang mag-eksperimento sa pag-aadjust ng laki ng minibatch upang mapabilis ang pagsasanay.

> **Note**: Ang ilang bersyon ng NVidia drivers ay kilalang hindi nagre-release ng memory pagkatapos sanayin ang modelo. Tumatakbo kami ng ilang halimbawa sa notebook na ito, at maaaring magdulot ito ng pagkaubos ng memory sa ilang setup, lalo na kung gumagawa ka ng sarili mong mga eksperimento bilang bahagi ng parehong notebook. Kung makaranas ka ng kakaibang mga error kapag sinimulan mong sanayin ang modelo, maaaring gusto mong i-restart ang kernel ng notebook.


In [3]:
batch_size = 16
embed_size = 64

## Simpleng RNN classifier

Sa kaso ng isang simpleng RNN, ang bawat recurrent unit ay isang simpleng linear network na tumatanggap ng input vector at state vector, at gumagawa ng bagong state vector. Sa Keras, ito ay maaaring i-representa gamit ang `SimpleRNN` layer.

Bagama't maaari nating ipasa ang one-hot encoded tokens nang direkta sa RNN layer, hindi ito magandang ideya dahil sa kanilang mataas na dimensionality. Kaya, gagamit tayo ng embedding layer upang bawasan ang dimensionality ng word vectors, susundan ng isang RNN layer, at sa huli ay isang `Dense` classifier.

> **Note**: Sa mga kaso kung saan hindi masyadong mataas ang dimensionality, halimbawa kapag gumagamit ng character-level tokenization, maaaring may saysay na ipasa ang one-hot encoded tokens nang direkta sa RNN cell.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Note:** Gumagamit tayo ng untrained embedding layer dito para sa kasimplehan, ngunit para sa mas magagandang resulta, maaari tayong gumamit ng pretrained embedding layer gamit ang Word2Vec, tulad ng tinalakay sa nakaraang unit. Magandang pagsasanay para sa iyo na i-adapt ang code na ito upang gumana gamit ang pretrained embeddings.

Ngayon, mag-train tayo ng ating RNN. Sa pangkalahatan, mahirap i-train ang RNNs dahil kapag ang mga RNN cells ay na-unroll sa haba ng sequence, ang bilang ng mga layers na kasangkot sa backpropagation ay nagiging napakarami. Kaya't kailangan nating pumili ng mas maliit na learning rate, at i-train ang network sa mas malaking dataset upang makakuha ng magagandang resulta. Maaari itong tumagal ng mahabang oras, kaya mas mainam na gumamit ng GPU.

Upang mapabilis ang proseso, itetrain lamang natin ang RNN model sa mga pamagat ng balita, at hindi isasama ang mga deskripsyon. Maaari mong subukang mag-train gamit ang deskripsyon at tingnan kung magagawa mong i-train ang model.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Tandaan** na ang katumpakan ay malamang na mas mababa dito, dahil nagsasanay lamang tayo sa mga pamagat ng balita.


## Muling Pagbalik sa mga Variable na Sequence

Tandaan na ang `TextVectorization` layer ay awtomatikong magdadagdag ng pad tokens sa mga sequence na may iba't ibang haba sa isang minibatch. Lumalabas na ang mga token na ito ay kasali rin sa training, at maaari nitong gawing mas mahirap ang convergence ng modelo.

May ilang mga paraan na maaari nating gawin upang mabawasan ang dami ng padding. Isa sa mga ito ay ang muling pag-aayos ng dataset batay sa haba ng sequence at pagsasama-sama ng lahat ng sequence ayon sa laki. Magagawa ito gamit ang `tf.data.experimental.bucket_by_sequence_length` function (tingnan ang [dokumentasyon](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Isa pang paraan ay ang paggamit ng **masking**. Sa Keras, may ilang mga layer na sumusuporta sa karagdagang input na nagpapakita kung aling mga token ang dapat isama sa training. Upang maisama ang masking sa ating modelo, maaari tayong magdagdag ng hiwalay na `Masking` layer ([docs](https://keras.io/api/layers/core_layers/masking/)), o maaari nating tukuyin ang `mask_zero=True` na parameter ng ating `Embedding` layer.

> **Note**: Ang training na ito ay tatagal ng humigit-kumulang 5 minuto upang makumpleto ang isang epoch sa buong dataset. Malaya kang ihinto ang training anumang oras kung nawawalan ka na ng pasensya. Maaari mo ring bawasan ang dami ng data na gagamitin para sa training sa pamamagitan ng pagdaragdag ng `.take(...)` clause pagkatapos ng `ds_train` at `ds_test` datasets.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Ngayon na gumagamit na tayo ng masking, maaari na nating sanayin ang modelo gamit ang buong dataset ng mga pamagat at deskripsyon.

> **Note**: Napansin mo ba na ginagamit natin ang vectorizer na sinanay sa mga pamagat ng balita, at hindi sa buong katawan ng artikulo? Posibleng magdulot ito ng pagkaligtaan ng ilang mga token, kaya mas mainam na muling sanayin ang vectorizer. Gayunpaman, maaaring maliit lamang ang epekto nito, kaya mananatili tayo sa naunang pre-trained na vectorizer para sa kasimplehan.


## LSTM: Long short-term memory

Isa sa mga pangunahing problema ng RNNs ay ang **vanishing gradients**. Ang RNNs ay maaaring maging mahaba, at maaaring mahirapan sa pagpapasa ng gradients pabalik sa unang layer ng network habang nagaganap ang backpropagation. Kapag nangyari ito, hindi matututo ang network ng mga relasyon sa pagitan ng mga malalayong token. Isang paraan upang maiwasan ang problemang ito ay ang pagpapakilala ng **explicit state management** gamit ang **gates**. Ang dalawang pinakakaraniwang arkitektura na gumagamit ng gates ay ang **long short-term memory** (LSTM) at **gated relay unit** (GRU). Tatalakayin natin ang LSTMs dito.

![Larawan na nagpapakita ng halimbawa ng long short term memory cell](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Ang isang LSTM network ay nakaayos sa paraang katulad ng RNN, ngunit may dalawang estado na ipinapasa mula layer patungo sa susunod: ang aktwal na estado $c$, at ang nakatagong vector $h$. Sa bawat unit, ang nakatagong vector $h_{t-1}$ ay pinagsasama sa input $x_t$, at magkasama nilang kinokontrol kung ano ang mangyayari sa estado $c_t$ at output $h_{t}$ sa pamamagitan ng **gates**. Ang bawat gate ay may sigmoid activation (output sa saklaw na $[0,1]$), na maaaring isipin bilang isang bitwise mask kapag pinarami sa state vector. Ang LSTMs ay may mga sumusunod na gates (mula kaliwa hanggang kanan sa larawan sa itaas):
* **forget gate** na tumutukoy kung aling mga bahagi ng vector $c_{t-1}$ ang kailangan nating kalimutan, at alin ang dapat ipasa.
* **input gate** na tumutukoy kung gaano karaming impormasyon mula sa input vector at nakaraang hidden vector ang dapat isama sa state vector.
* **output gate** na kumukuha ng bagong state vector at nagpapasya kung aling mga bahagi nito ang gagamitin upang makabuo ng bagong hidden vector $h_t$.

Ang mga bahagi ng estado $c$ ay maaaring isipin bilang mga flag na maaaring i-on o i-off. Halimbawa, kapag nakita natin ang pangalang *Alice* sa isang sequence, mahihinuha natin na tumutukoy ito sa isang babae, at itataas ang flag sa estado na nagsasabing mayroong pangngalang pambabae sa pangungusap. Kapag nakita pa natin ang mga salitang *and Tom*, itataas natin ang flag na nagsasabing mayroong pangngalang maramihan. Sa ganitong paraan, sa pamamagitan ng pagmamanipula ng estado, maaari nating subaybayan ang mga gramatikal na katangian ng pangungusap.

> **Note**: Narito ang isang mahusay na mapagkukunan para maunawaan ang mga detalye ng LSTMs: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) ni Christopher Olah.

Bagama't maaaring mukhang kumplikado ang panloob na istruktura ng isang LSTM cell, itinatago ng Keras ang implementasyong ito sa loob ng `LSTM` layer, kaya ang tanging kailangan nating gawin sa halimbawa sa itaas ay palitan ang recurrent layer:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Bidirectional at Multilayer na RNNs

Sa mga halimbawa natin hanggang ngayon, ang mga recurrent network ay gumagana mula sa simula ng isang sequence hanggang sa dulo. Natural ito para sa atin dahil sumusunod ito sa parehong direksyon kung paano tayo nagbabasa o nakikinig ng pananalita. Gayunpaman, para sa mga sitwasyon na nangangailangan ng random na pag-access sa input sequence, mas may saysay na patakbuhin ang recurrent computation sa parehong direksyon. Ang mga RNN na nagpapahintulot ng computation sa parehong direksyon ay tinatawag na **bidirectional** RNNs, at maaari itong malikha sa pamamagitan ng pag-wrap sa recurrent layer gamit ang espesyal na `Bidirectional` layer.

> **Note**: Ang `Bidirectional` layer ay gumagawa ng dalawang kopya ng layer sa loob nito, at itinatakda ang `go_backwards` na property ng isa sa mga kopyang iyon sa `True`, kaya't ito ay gumagalaw sa kabaligtarang direksyon ng sequence.

Ang mga recurrent network, unidirectional man o bidirectional, ay kumukuha ng mga pattern sa loob ng isang sequence, at iniimbak ang mga ito sa state vectors o ibinabalik bilang output. Tulad ng convolutional networks, maaari tayong magtayo ng isa pang recurrent layer kasunod ng una upang makuha ang mas mataas na antas ng mga pattern, na binuo mula sa mas mababang antas ng mga pattern na nakuha ng unang layer. Ito ang nagdadala sa atin sa konsepto ng isang **multi-layer RNN**, na binubuo ng dalawa o higit pang recurrent networks, kung saan ang output ng nakaraang layer ay ipinapasa sa susunod na layer bilang input.

![Larawan na nagpapakita ng isang Multilayer long-short-term-memory- RNN](../../../../../translated_images/tl/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Larawan mula sa [napakagandang post na ito](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) ni Fernando López.*

Ginagawang madali ng Keras ang pagbuo ng mga network na ito, dahil kailangan mo lang magdagdag ng mas maraming recurrent layers sa model. Para sa lahat ng layers maliban sa huli, kailangan nating tukuyin ang parameter na `return_sequences=True`, dahil kailangan natin ang layer na ibalik ang lahat ng intermediate states, at hindi lang ang final state ng recurrent computation.

Gumawa tayo ng isang two-layer bidirectional LSTM para sa ating classification problem.

> **Note** ang code na ito ay muling nangangailangan ng mahabang oras upang matapos, ngunit nagbibigay ito ng pinakamataas na accuracy na nakita natin hanggang ngayon. Kaya marahil sulit itong hintayin at tingnan ang resulta.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs para sa iba pang mga gawain

Hanggang ngayon, nakatuon tayo sa paggamit ng RNNs para magklasipika ng mga teksto. Ngunit kaya rin nilang magamit sa maraming iba pang gawain, tulad ng pagbuo ng teksto at pagsasalin ng wika — tatalakayin natin ang mga gawain na ito sa susunod na unit.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
